## Introduction to Path Analysis using Batfish

Path analysis is one of the most common tasks a network engineer will undertake, but one of the most complicated ones. Traditionally, path analysis is performed by using `traceroute`. In a lot of instances, this must be performed from multiple locations in the network. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-use queries.

In this notebook, we will look at how you can perform path analysis with Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-path-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py
bf_session.additionalArgs['debugflags']='traceroute'

/Users/victor/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Find the path taken by AS3 core routers to reach `host1`, the DNS Server in AS2

To do this, we will use the [`traceroute` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.traceroute) in Batfish. The question has three (composite) parameters that you can specify, allowing for a variety of queries. We will focus on the two main ones:

* `startLocation` - a place in the network where a trace can start
* `headers` - IPv4 packet headers that allow you to craft a packet with which to do a traceroute (**not** just limited to UDP or ICMP)

We want the query to start from the `Loopback0` interface on `as3core1`, and we want to use the IP address of that interface as the source address.
For this we set the `startLocation` to `as3core1[Loopback0]` 
Note that Batfish automatically chooses the source IP address of `Loopback0` as the source IP for our packet.

Lets set the destination IP address of our virtual packet by specifying `dstIps = "ofLocation(host1)"`. Batfish will automatically pick *one of the* IP address for `host1` as the destination IP address for the question.

To run the query:

In [3]:
# start the traceroute from the Loopback0 interface of as3core1 to host1
tracert = bfq.traceroute(startLocation="as3core1[Loopback0]", headers=HeaderConstraints(dstIps='ofLocation(host1)')).answer().frame()

To pretty-print the query in HTML use the `display_html` function. We will show you how to extract more detailed information below

In [4]:
display_html(tracert)

The results are interpreted as follows: 
A flow that starts at `as3core` going from `3.10.1.1:49152` to `2.128.0.101:33434` (UDP by default) and has 4 traces (i.e., paths), with 7 hops each.

Note that compared to running traceroute from an actual node in your network, Batfish returns additional information:
1. All active parallel paths between the source and destination
2. The reason why each hop in a path is taken (the specific routing entry that was matched)
3. Disposition of the packet for each path

Now let's take a look at one of those paths in more detail. Let's start with the first path:

In [5]:
# flow number we want to see
flow_num = 0
# path number we want to see
path_num = 0
# print the hops for the first trace
path = tracert['Traces'][flow_num][path_num]
display(path)

Trace(disposition='DENIED_IN', hops=[Hop(node='as3core1', steps=[Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/16', 'nextHopIp': '10.23.21.2', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as3border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/16', 'nextHopIp': '10.23.21.2', 'protocol': 'bgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2border2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter='OUTSIDE_TO_INSIDE'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'ibgp'}, {'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'bgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default', inputFilter='filter::INPUT'), action='DENIED')])])

What we see from the above output is that the packet was sent all the way to the host, but blocked at the host itself (hop 7).

The packet was `DENIED`, which means that a filter on the host denied the packet. 
Let's determine the name of the filter that matched the packet by calling `disposition_reason()` on the path object:

This tells us that the incoming filter on `host1` on inteface `eth0` blocked the packet.

## Abstract path analysis (reachability)

Traceroute allows you to find the paths taken by a specified flow through the network, however Batfish allows you to run even more powerful queries.
[`reachability` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.reachability) 
will allow you to explore the network in a more abstract fashion. 

This is very useful 
when you want to build a set of tests for the network to ensure security and reliability.

Lets start by constructing a query that checks if **any** flow from the IPv4 address space (`srcIps='0.0.0.0/0`) orignating inside AS2 would will be able to reach `host1`.

In [6]:
path = PathConstraints(startLocation="as2.*")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", ipProtocols="TCP,UDP")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="accepted").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount
0,0.0.0.0:0 → 2.128.0.101:22start=as2border1,"ACCEPTED1. node: as2border1steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet1/0)2. node: as2core1steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]) → TRANSMITTED(GigabitEthernet2/0)3. node: as2dist1steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]) → TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)5. node: host1steps: RECEIVED(eth0: filter::INPUT) → ACCEPTED(InboundStep)ACCEPTED1. node: as2border1steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet1/0)2. node: as2core1steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)5. node: host1steps: RECEIVED(eth0: filter::INPUT) → ACCEPTED(InboundStep)ACCEPTED1. node: as2border1steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)2. node: as2core2steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)3. node: as2dist2steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)5. node: host1steps: RECEIVED(eth0: filter::INPUT) → ACCEPTED(InboundStep)ACCEPTED1. node: as2border1steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)2. node: as2core2steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]) → TRANSMITTED(GigabitEthernet3/0)3. node: as2dist1steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4]) → TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)5. node: host1steps: RECEIVED(eth0: filter::INPUT) → ACCEPTED(InboundStep)",4
1,0.0.0.0:0 → 2.128.0.101:22start=as2border2,"ACCEPTED1. node: as2border2steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet1/0)2. node: as2core2steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)3. node: as2dist2steps: RECEIVED(GigabitEthernet0/0) → FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) → TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1steps: RECEIVED(GigabitEthernet1/0) → FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) → TRANSMITTED(GigabitEthernet2/0)5. node: host1steps: RECEIVED(eth0: filter::INPUT) → ACCEPTED(InboundStep)ACCEPTED1. node: as2border2steps: FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.101.4],ibgp 

As you can see, this query identified flows entering the network at *all* `as2...` nodes destined for `host1` that would be delivered.
And in all cases, it was SSH traffic that was permitted. However, such output can be hard to process, therefore 
with reachability queries we will want to create queries that are the **opposite** of the property we want to quarantee.

For example, let's check if any check if *any* DNS flow originating *inside* AS2 destined for `host1` is *dropped*

In [7]:
path = PathConstraints(startLocation="as2.*")
headers = HeaderConstraints(dstIps="ofLocation(host1)", applications="SSH")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount


The fact that we get empty result means that `host1` is reachable via SSH from anywhere within AS2.
Let's now do the same for DNS traffic:

Check if *any* DNS flow originating *outside* of AS2 destined for `host1` is *dropped*
## TODO: explain enter grammar

In [8]:
path = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", applications="DNS")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount
0,2.0.0.0:0 → 2.128.0.101:53start=as2border1 interface=GigabitEthernet0/0,DENIED_IN1. node: as2border1steps: DENIED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE),1
1,2.0.0.0:0 → 2.128.0.101:53start=as2border2 interface=GigabitEthernet0/0,DENIED_IN1. node: as2border2steps: DENIED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE),1


The answer is yes! That means that at least `2.0.0.0` cannot reach the DNS server due to the `OUTSIDE_TO_INSIDE` filter.

Finally, let's perform a security check, searching for any flows **accepted** by `host1` that are **not** DNS or SSH

In [9]:
path = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", ipProtocols="TCP,UDP", dstPorts="!22,!53")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="accepted").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount


Sucess! No unauthorized (DNS or SSH) flows will reach `host1`

### Wrap-up

This concludes the notebook. To recap, in this notebook we covered the foundational tasks for path analysis:

1. We performed a traceroute to check connectivity to `host1`
2. Analyzed detailed path & hop information for the traceroute
3. Explored a space of flows with the reachablity question and found a ACL bug that prevents some clients from reaching the DNS server
4. Perfomed a security check that ensures that only SSH and DNS traffic can reach `host1`

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics ensuring planned configuration changes do not have unintended consequences. Stay tuned!

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.